### Feature engineering - CategoryEncoding - OneHotEncoder
OneHotEncoder with sklearn on the Titanic dataset 

***
#### Environment
`conda activate sklearn-env`

***
#### Goals
- Replace categorical and discrete columns with categorical elements corespondng to each value from dataset
- Note that name is not identified as a categorical value

***
#### References
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html


#### Basic python imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

#### Dataset load using sklearn API from https://www.openml.org site

https://www.openml.org/d/40945

In [ ]:
from sklearn.datasets import fetch_openml

# Load data from https://www.openml.org/d/40945
raw_dataset = fetch_openml("titanic", version=1, as_frame=True).frame
dataset = raw_dataset.copy()
dataset.head(10)

In [ ]:
dataset.drop(['boat', 'body', 'home.dest'],  axis=1, inplace=True)
dataset = dataset.dropna().copy()
dataset.reset_index(drop=True, inplace=True)

### Verify categorical /discrete fields in dataset

Notice:
- `pclass` discrete fiels having 3 distinct values.
- `sex` and `embarked` fields - categorical

In [ ]:
dataset.info()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder().fit(dataset[['pclass', 'sex', 'embarked']])
encoder.categories_

In [ ]:
ohe_f = encoder.transform(dataset[['pclass', 'sex', 'embarked']])
print(ohe_f)

In [ ]:
ohe_dataset = pd.DataFrame.sparse.from_spmatrix(ohe_f)
ohe_dataset.columns =['pclass_1', 'pclass_2', 'pclass_3', 'sex_F', 'sex_M', 'embarked_C', 'embarked_Q', 'embarked_S']

# the 2 datasets must have the same indexes or this operation will introduce NaNs
new_dataset = pd.concat([dataset, ohe_dataset], axis=1)

new_dataset.sample(7)